In [247]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [248]:
Title = []
release_year = []
certificate = []
duration = []
genre = []
rating = []
metascore = []
director = []
cast = []
votes = []
desc = []

In [249]:
for i in range(1, 201):
    url = f"https://www.imdb.com/search/keyword/?mode=detail&page={i}&ref_=kw_ref_typ&title_type=movie&sort=moviemeter,asc"
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')

    data = soup.findAll('div', class_='lister-item mode-detail')

    for movie in data:

        try:
            name = movie.h3.a.get_text()
            Title.append(name)
        except AttributeError:
            Title.append(None)

        try:
            cRate = movie.find('span', class_='certificate').get_text()
            certificate.append(cRate)
        except AttributeError:
            certificate.append(None)

        try:
            runtime = movie.find('span', class_='runtime').get_text().replace(' min', '')
            duration.append(runtime)
        except AttributeError:
            duration.append(None)

        try:
            year = data[33].find('span', class_='lister-item-year text-muted unbold').get_text()
            release_year.append(re.search(r'\b\d{4}\b', year).group())
        except AttributeError:
            release_year.append(None)

        try:
            genres = ', '.join([movie.find('span', class_='genre').get_text(strip=True)])
            genre.append(genres)
        except AttributeError:
            genre.append(None)

        try:
            rate = movie.find('div', class_='inline-block ratings-imdb-rating').get_text(strip=True)
            rating.append(rate)
        except AttributeError:
            rating.append(None)

        try:
            score = movie.find('div', class_='inline-block ratings-metascore').span.get_text(strip=True)
            metascore.append(score)
        except AttributeError:
            metascore.append(None)

        try:
            crew = movie.find_all('p', class_='text-muted text-small')[1]
            try:
                Dir = crew.find('a').get_text(strip=True)
                director.append(Dir)
            except AttributeError:
                director.append(None)
            try:
                stars = ', '.join([a.get_text(strip=True) for a in crew.find_all('a')[1:]])
                cast.append(stars)
            except AttributeError:
                cast.append(None)
        except AttributeError:
            director.append(None)
            cast.append(None)

        try:
            nv = movie.find('span', attrs={'name':'nv'})
            votes.append(nv.text)
        except AttributeError:
            votes.append(None)

        try:
            summary = movie.find('p',class_='').get_text(strip=True)
            desc.append(summary)
        except AttributeError:
            desc.append(None)


In [250]:
movies = pd.DataFrame({
    'Title' : Title,
    'Year': release_year,
    'Certificate': certificate,
    'Duration (min)': duration,
    'Genre': genre,
    'Rating': rating,
    'Metascore': metascore,
    'Director': director,
    'Cast': cast,
    'Votes': votes,
    'Description': desc
})
movies

,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description
0,The Idea of You,2023,R,115,"Comedy, Drama, Romance",6.4,67,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","27,779","Solène, a 40-year-old single mom, begins an un..."
1,Kingdom of the Planet of the Apes,2023,PG-13,145,"Action, Adventure, Sci-Fi",7.3,66,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","20,257","Many years after the reign of Caesar, a young ..."
2,Unfrosted,2023,PG-13,97,"Biography, Comedy, History",5.5,42,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","17,823","In 1963 Michigan, business rivals Kellogg's an..."
3,The Fall Guy,2023,PG-13,126,"Action, Comedy, Drama",7.3,73,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","37,767",A down-and-out stuntman must find the missing ...
4,Challengers,2023,R,131,"Drama, Romance, Sport",7.7,82,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","31,809","Tashi, a former tennis prodigy turned coach, t..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,The Greatest Show on Earth,2020,U,152,"Drama, Family, Romance",6.5,76,Cecil B. DeMille,"James Stewart, Charlton Heston, Betty Hutton, ...","16,076","The dramatic lives of trapeze artists, a clown..."
9996,Berserk: Ougon Jidai-hen I - Haou no Tamago,2020,None,76,"Animation, Action, Adventure",7.5,None,Toshiyuki Kubooka,"Hiroaki Iwanaga, Carrie Keranen, Takahiro Saku...","14,299",A lone sellsword named Guts gets recruited int...
9997,Is-slottet,2020,None,78,"Mystery, Drama",6.5,None,Per Blom,"Line Storesund, Hilde Nyeggen Martinsen, Meret...",740,A couple of twelve-year-old Norwegian girls st...
9998,Loving Pablo,2020,A,123,"Biography, Crime, Drama",6.4,42,Fernando León de Aranoa,"Javier Bardem, Penélope Cruz, Peter Sarsgaard,...","22,443",A journalist strikes up a romantic relationshi...


In [253]:
movies.to_csv("data_scrapped/imdb-movies-dataset.csv", index=False)